# NYC Taxi Dataset Analysis
This notebook uses PySpark to analyze the NYC Yellow Taxi dataset for January 2018.
Ensure you have PySpark installed locally to run this notebook.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('NYC Taxi Analysis').getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
df = spark.read.parquet('data/yellow_tripdata_2018-01.parquet')
df.printSchema()
df.show(5)

In [ ]:
# Query 1 - Add Revenue Column
from pyspark.sql.functions import col
df = df.withColumn(
    'Revenue',
    col('fare_amount') + col('extra') + col('mta_tax') +
    col('improvement_surcharge') + col('tip_amount') +
    col('tolls_amount') + col('total_amount')
)
df.select('Revenue').show(5)

In [ ]:
# Query 2 - Passenger count by area
df.groupBy('PULocationID').sum('passenger_count').withColumnRenamed('sum(passenger_count)', 'total_passengers').show()

In [ ]:
# Query 3 - Average fare / earnings by vendor
from pyspark.sql.functions import avg, round
df.groupBy('VendorID').agg(
    round(avg('fare_amount'), 2).alias('avg_fare'),
    round(avg('total_amount'), 2).alias('avg_earning')
).show()

In [ ]:
# Query 4 - Count by payment mode
from pyspark.sql.functions import count
df.groupBy('payment_type').agg(count('*').alias('payment_count')).show()

In [ ]:
# Query 5 - Top 2 vendors on a specific date
from pyspark.sql.functions import to_date, sum
df = df.withColumn('trip_date', to_date('tpep_pickup_datetime'))
df.filter(df.trip_date == '2018-01-15') \
  .groupBy('VendorID') \
  .agg(sum('passenger_count').alias('total_passengers'),
       sum('trip_distance').alias('total_distance'),
       sum('total_amount').alias('total_earning')) \
  .orderBy(col('total_earning').desc()) \
  .limit(2).show()

In [ ]:
# Query 6 - Route with most passengers
df.groupBy('PULocationID', 'DOLocationID') \
  .sum('passenger_count') \
  .orderBy(col('sum(passenger_count)').desc()) \
  .show(1)

In [ ]:
# Query 7 - Top pickup locations in last 10 seconds (simulated)
from pyspark.sql.functions import max, unix_timestamp, lit
max_time = df.agg(max('tpep_pickup_datetime')).collect()[0][0]
filtered_df = df.filter(
    unix_timestamp('tpep_pickup_datetime') >= unix_timestamp(lit(max_time)) - 10
)
filtered_df.groupBy('PULocationID').sum('passenger_count').orderBy(col('sum(passenger_count)').desc()).show()